<a href="https://colab.research.google.com/github/SeokcheonMoon/study_data_analytics/blob/main/docs/quests/CDAs/CDA_mixed_unVar_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime 이후 상단바 런타임의 Restart Session 을 클릭해야함
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
df_fruitstore = pd.read_csv("/content/drive/MyDrive/fruitstore_saleshistory.csv")
df_fruitstore

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),1.0,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08
4,2018-01-02,화,1,1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,수지구,풍덕천동,여,34.0,30대이하,4403,매장C,NaN,10:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705566,2018-06-30,토,6,26,146297344,정상회원,수산,건어물,멸치류,국물멸치(300g),1.0,수지구,성복동,여,51.0,50대,9189,매장A,NaN,17:56
705567,2018-06-30,토,6,26,1532985472,정상회원,수산,해조,미역/다시마,자른다시마(100g),1.0,수지구,동천동,여,63.0,60대,5169,매장A,NaN,14:37
705568,2018-06-30,토,6,26,1532985472,정상회원,반찬,두부/유부,두부,두부(1모:420g),2.0,수지구,동천동,여,63.0,60대,7466,매장A,NaN,14:37
705569,2018-06-30,토,6,26,1209779776,정상회원,간식,과자,스낵,하늘바다새우(60g),1.0,수지구,동천동,여,41.0,40대,3542,매장A,NaN,18:04


## 가설 : 성별(범주형)에 따라 구매한 상품의 단가(연속형) 분산이 같은가?

- 귀무가설 : 성별에 따른 구매 상품의 단가 분산은 같지 않다.(성별과 구매 금액은 관련이 없다.)
- 대립가설 : 성별에 따른 구매 상품의 단가 분산은 같다.(성별과 구매 금액은 관련이 있다.)

### 정규/비정규 분포 판단 - Anderson test

In [ ]:
# 레코드 갯수가 5천개 이상 Anderson test
stats.anderson(df_fruitstore["구매금액"])

AndersonResult(statistic=80887.23714091501, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=10920.038082630947, scale=16703.23407010683)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

statistic > critical_values 이므로  => 비정규분포를 따른다.

### 범주(성별)가 2가지(남,여) - Wilcoxon test

In [ ]:
condition_male = "성별=='남'"
condition_female = "성별=='여'"

In [ ]:
series_male = df_fruitstore.query(condition_male)["구매금액"]
series_male

30        16273
34         3829
38        20676
71         7083
90        16081
          ...  
705445     4020
705446     9189
705481     2967
705502    49775
705542     7466
Name: 구매금액, Length: 43542, dtype: int64

In [ ]:
series_female = df_fruitstore.query(condition_female)["구매금액"]
series_female

0         22207
1          4977
2          7083
3           766
4          4403
          ...  
705566     9189
705567     5169
705568     7466
705569     3542
705570     1914
Name: 구매금액, Length: 662024, dtype: int64

In [23]:
stats.ranksums(series_male,series_female)

RanksumsResult(statistic=1.571683301008152, pvalue=0.11602401682141163)

## 결과
- 통계결과 : pvalue > 0.05 -> 귀무가설에 일치
- 분석결과 : 성별에 따른 구매 상품의 단가 분산은 같지 않다. (성별과 구매 금액은 관련이 없다.)